In [1]:
from hypernetworks.core.Hypernetwork import Hypernetwork
from hypernetworks.core.Hypersimplex import VERTEX, ALPHA, BETA, PROPERTY, UNION_ALPHA, IMMUTABLE_ALPHA, SEQUENCE, hstype_to_str
from hypernetworks.utils.HTCompiler import load_parser, compile_hn
from hypernetworks.utils.HTGraph import draw_hn
from hypernetworks.utils.HTTools import get_subHn_by_semantic_boundary

from IPython.display import Image, display

from graphviz import nohtml

import graphviz
import pprint as pp
import re
import textwrap
import iplantuml

pictures_dir = "pictures/TheCo/"
parser = load_parser()

In [2]:
capabilities_hn = Hypernetwork()

compile_hn(capabilities_hn, parser, """
    job-booking=<; R_capability; B(capabilities)>
    scheduling=<; R_capability; B(capabilities)>
    job-management=<dispatch, deprogram; R_capability; B(capabilities)>
        dispatch=<; R_capability; B(capabilities)>
        deprogram=<; R_capability; B(capabilities)>
        
    field-management=<; R_capability; B(capabilities)>
    field-operations=<; R_capability; B(capabilities)>
    asset-management=<; R_capability; B(capabilities)>

    billing=<; R_capability; B(capabilities)>

    externals={customer, internal, external; R_externals; B(external)}
""", boundary_percolation=True)

# draw_hn(capabilities_hn, fname=pictures_dir+"capabilities-hn", view=False, show_hstype=False, show_vertex=False)
# Image(filename=pictures_dir+"capabilities-hn.png", width=1000, height=1200)

In [3]:
business_objects_hn = Hypernetwork()

compile_hn(business_objects_hn, parser, """
    business-objects={job, job-result, asset, contract; R_business-objects}
""")

# draw_hn(business_objects_hn, fname=pictures_dir+"business-objects-hn", view=False, show_hstype=False, show_vertex=False)
# Image(filename=pictures_dir+"business-objects-hn.png", width=200, height=1200)

In [4]:
capability_flow_hn = Hypernetwork()

compile_hn(capability_flow_hn, parser, """
    customer-job-booking=<customer, job, job-booking; R_flow; B(capability-flows)>
    internal-job-booking=<internal, job, job-booking; R_flow; B(capability-flows)>
    external-job-booking=<external, job, job-booking; R_flow; B(capability-flows)>

    job-booking-scheduling=<job-booking, job, scheduling; R_flow; B(capability-flows)>
    job-booking-dispatch=<job-booking, job, dispatch; R_flow; B(capability-flows)>

    asset-management-dispatch=<asset-management, asset, dispatch; R_flow; B(capability-flows)>
    dispatch-field-management=<dispatch, <job, asset>, field-management; R_flow; B(capability-flows)>
    dispatch-field-operations=<dispatch, <job, asset>, field-operations; R_flow; B(capability-flows)>

    field-operations-deprogram=<field-operations, deprogram; R_flow; B(capability-flows)>

    deprogram-billing=<deprogram, job-result, billing; R_flow; B(capability-flows)>
    deprogram-asset-management=<deprogram, asset, asset-management; R_flow; B(capability-flows)>
    deprogram-customer=<deprogram, job-result, customer; R_flow; B(capability-flows)>
    deprogram-external=<deprogram, job-result, external; R_flow; B(capability-flows)>
    deprogram-internal=<deprogram, job-result, internal; R_flow; B(capability-flows)>
""", boundary_percolation=True)

# draw_hn(capability_flow_hn, fname=pictures_dir+"capability-flow-hn", view=False, show_hstype=False, show_vertex=False, direction="LR")
# Image(filename=pictures_dir+"capability-flow-hn.png", width=900, height=1200)

In [5]:
hn = Hypernetwork()

hn.copy(capabilities_hn)
hn.merge(business_objects_hn)
hn.merge(capability_flow_hn)

# draw_hn(hn, fname=pictures_dir+"hn", view=False, show_hstype=False, direction="LR")
# Image(filename=pictures_dir+"hn.png", width=1000, height=1200)

In [6]:
old_applications_hn = Hypernetwork()

compile_hn(old_applications_hn, parser, """
    old-applications={
        JB=<job-booking; R_system; B(old, system)>,
        Sched=<scheduling; R_system; B(old, system)>,
        AM=<asset-management; R_system; B(old, system)>,
        Integration=<job-management; R_system; B(old, system)>,
        Field=<field-management, field-operations; R_system; B(old, system)>,
        Billing=<billing; R_system; B(old, system)>
    ; R_applications; B(applications)}
""")

# draw_hn(old_applications_hn, fname=pictures_dir+"old-applications-hn", view=False, show_hstype=False, show_vertex=False)
# Image(filename=pictures_dir+"old-applications-hn.png", width=900, height=1200)

In [7]:
new_applications_hn = Hypernetwork()

compile_hn(new_applications_hn, parser, """
    new-applications={
        CRM=<job-booking; R_system; B(new, system)>,
        FM-AM=<scheduling, asset-management, job-management, field-management, field-operations; R_system; B(new, system)>,
        Billing=<billing; R_system; B(new, system)>
    ; R_applications; B(applications)}
""")

# draw_hn(new_applications_hn, fname=pictures_dir+"new-applications-hn", view=False, show_hstype=False, show_vertex=False)
# Image(filename=pictures_dir+"new-applications-hn.png", width=800, height=1200)

In [8]:
hn = Hypernetwork()

hn.copy(old_applications_hn)
hn.merge(new_applications_hn)

# draw_hn(hn, fname=pictures_dir+"new-old-applications-hn", view=False, show_hstype=False, show_vertex=False)
# Image(filename=pictures_dir+"new-old-applications-hn.png", width=1500, height=1200)

In [9]:
def satisfied_by(hn, value):
    if value not in hn.hypernetwork:
        print(value, "not found.")
        return None

    print(value, "is satisfiled by these applications:")
    
    for part in hn.hypernetwork[value].partOf:
        print("\t", part, "in boundaries:", ", ".join(hn.hypernetwork[part].B))

In [10]:
satisfied_by(hn, "scheduling")
print()
satisfied_by(hn, "job-booking")

scheduling is satisfiled by these applications:
	 Sched in boundaries: old, system
	 FM-AM in boundaries: new, system

job-booking is satisfiled by these applications:
	 JB in boundaries: old, system
	 CRM in boundaries: new, system


In [11]:
hn.merge(capability_flow_hn)

print(hn.get_boundaries())

# draw_hn(hn, fname=pictures_dir+"new-old-applications-flow-hn", view=False, show_hstype=False, show_vertex=False, direction="LR")
# Image(filename=pictures_dir+"new-old-applications-flow-hn.png", width=1000, height=1200)

{'old', 'new', 'capability-flows', 'system', 'applications'}


In [12]:
satisfied_by(hn, "scheduling")

scheduling is satisfiled by these applications:
	 Sched in boundaries: old, system
	 FM-AM in boundaries: new, system
	 job-booking-scheduling in boundaries: capability-flows


In [17]:
old_Hn = Hypernetwork()

old_Hn.copy(business_objects_hn)
old_Hn.merge(capabilities_hn)
old_Hn.merge(capability_flow_hn)

print(old_Hn.hypernetwork["job-management"])
old_Hn.merge(old_applications_hn)
print(old_Hn.hypernetwork["job-management"])

print(old_Hn.get_boundaries())
print(old_Hn.hypernetwork["customer"].B)

# draw_hn(old_Hn, fname=pictures_dir+"old-system-flows-hn", view=False, show_hstype=False, direction="LR")
# Image(filename=pictures_dir+"old-system-flows-hn.png", width=1000, height=1200)

job-management=<dispatch, deprogram; R_capability; B(capabilities)>^N
job-management=<dispatch, deprogram; R_capability; B(capabilities)>^N
{'old', 'external', 'capability-flows', 'system', 'applications', 'capabilities'}
{'external', 'capability-flows'}


In [14]:
new_Hn = Hypernetwork()

new_Hn.copy(business_objects_hn)
new_Hn.merge(capabilities_hn)
new_Hn.merge(capability_flow_hn)
new_Hn.merge(new_applications_hn)

print(new_Hn.get_boundaries())

# draw_hn(new_Hn, fname=pictures_dir+"new-system-flows-hn", view=False, show_hstype=False, direction="LR")
# Image(filename=pictures_dir+"new-system-flows-hn.png", width=1000, height=1200)

{'new', 'external', 'capability-flows', 'system', 'applications', 'capabilities'}


In [15]:
from graphviz import Digraph

PROTOCOL = 1
LABEL = 3

def generate_directed_graph(hn, label_pos=1, output_file='output_graph', dir="TD"):
    def _hn_to_triples(label_pos=1):
        triples = []

        for name, hs in hn.hypernetwork.items():
            label = ""

            if hs.hstype not in [VERTEX]:
                if "flow" == hs.R.name:
                    start = end = None

                    for v in hs.simplex:
                        for p in hn.hypernetwork[v].partOf:
                            if "system" in hn.hypernetwork[p].B:
                                if start == None:
                                    start = hn.hypernetwork[p].vertex
                                
                                elif end == None:
                                    end = hn.hypernetwork[p].vertex

                        if "external" in hn.hypernetwork[v].B:
                            if start == None:
                                start = v
                            
                            elif end == None:
                                end = v

                    if hn.hypernetwork[hs.simplex[1]].hstype in [ALPHA]:
                        label = ", ".join(hn.hypernetwork[hs.simplex[1]].simplex)

                    else:
                        label = hs.simplex[1]
                        
                    triples.append((start, label, end, hs.R.name))
                    
        return triples
        
    dot = Digraph(engine="dot", graph_attr={"rankdir":dir})
    triples = _hn_to_triples(label_pos)

    dot.attr('node', shape='circle', fixedsize='true', width='1', height='1')

    for start_node, label, end_node, R in triples:
        start_label = start_node.replace("-", "<br/>")
        end_label = end_node.replace("-", "<br/>")

        wrapped_start_node = f'<<table border="0" cellpadding="3"><tr><td>{start_label}</td></tr></table>>'
        wrapped_end_node = f'<<table border="0" cellpadding="3"><tr><td>{end_label}</td></tr></table>>'

        dot.node(start_node, label=wrapped_start_node)
        dot.node(end_node, label=wrapped_end_node)
        dot.edge(start_node, end_node, label=label)  # Add edge with label
        
    dot.render(output_file, format='png', cleanup=True)

In [16]:
print(old_Hn.hypernetwork["job-management"])
generate_directed_graph(old_Hn, LABEL, pictures_dir+"old-systems")
Image(filename=pictures_dir+"old-systems.png", width=600)

job-management=<dispatch, deprogram; R_capability; B(capabilities)>^N


AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
generate_directed_graph(new_Hn, LABEL, pictures_dir+"new-systems")
Image(filename=pictures_dir+"new-systems.png", width=600)